In [1]:
# testing lib fn
from __future__ import print_function, unicode_literals
from unicodedata import normalize
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Masking
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import argparse
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_pickle('df1_pos_W.pkl')

In [3]:
pos_dict = lib.get_pos_dict(df)

In [4]:
df = lib.concat_word_pos(df, columns=['context_lemma_pos', 'question_lemma_pos','answers_lemma_pos'])

In [5]:
vocabulary = lib.get_total_vocab(df, columns=['context_lemma_pos', 'question_lemma_pos'])

In [6]:
df['text'] = df['context_lemma_pos'] + ' ' + df['question_lemma_pos']

In [7]:
df['question_answer'] = df['question_lemma_pos'] + ' ' + df['answers_lemma_pos']

In [8]:
df.head()

,context_lemma_pos,question_lemma_pos,answers_lemma_pos,answer_len,question_len,context_len,text,question_answer
2,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonce_NN leave_VBP destinys_VBN chi...,2003_CD,1,8,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonce_NN leave_VBP destinys_VBN chi...
11,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonc_NN release_NN dangerously_RB l...,2003_CD,1,5,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonc_NN release_NN dangerously_RB l...
12,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,how_WRB many_JJ grammy_JJ award_NNS beyonc_NN ...,five_CD,1,9,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,how_WRB many_JJ grammy_JJ award_NNS beyonc_NN ...
15,follow_VBG disbandment_NN destiny_NN child_NN ...,after_IN second_JJ solo_NN album_NN entertainm...,act_VBG,1,8,112,follow_VBG disbandment_NN destiny_NN child_NN ...,after_IN second_JJ solo_NN album_NN entertainm...
17,follow_VBG disbandment_NN destiny_NN child_NN ...,to_TO set_VB record_NN grammys_NN many_JJ beyo...,six_CD,1,7,112,follow_VBG disbandment_NN destiny_NN child_NN ...,to_TO set_VB record_NN grammys_NN many_JJ beyo...


In [9]:
train_data, valid_data, vocabulary, reversed_dictionary, vocabulary_size, embedding_matrix, embedding_dim = lib.load_sequential_data(df,
                                                                                                                                    pos_dict,
                                                                                                                                    shuffle_df=False,
                                                                                                                                    use_glove=True, 
                                                                                                                                    embedding_dim=300, 
                                                                                                                                    use_pos=True)

First five words  in vectorized format :  [113534, 33057, 13160, 73117, 74266]
Vocabulary examples : 
1.  cimas_VBP 55621
2.  nbl_JJ 29466
3.  konishi_NN 37825
 Size of vocabulary :  127983
beyonc_NN giselle_NN knowlescarter_NN bijnse_NN beeyonsay_VBP born_JJ september_NN 4_CD 1981_CD american_JJ singer_NN songwriter_NN record_NN producer_NN actress_NN born_NNS raise_VBD houston_NN texas_NN perform_VBD various_JJ sing_VBG dance_VBG competition_NNS child_NN rise_VBD fame_JJ late_RB 1990s_CD lead_JJ



In [10]:
embedding_dim

301

In [11]:
vocabulary_size

127983

In [12]:
len(train_data)

952856

In [13]:
df.head()

,context_lemma_pos,question_lemma_pos,answers_lemma_pos,answer_len,question_len,context_len,text,question_answer
0,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonce_NN leave_VBP destinys_VBN chi...,2003_CD,1,8,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonce_NN leave_VBP destinys_VBN chi...
1,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonc_NN release_NN dangerously_RB l...,2003_CD,1,5,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,when_WRB beyonc_NN release_NN dangerously_RB l...
2,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,how_WRB many_JJ grammy_JJ award_NNS beyonc_NN ...,five_CD,1,9,75,beyonc_NN giselle_NN knowlescarter_NN bijnse_N...,how_WRB many_JJ grammy_JJ award_NNS beyonc_NN ...
3,follow_VBG disbandment_NN destiny_NN child_NN ...,after_IN second_JJ solo_NN album_NN entertainm...,act_VBG,1,8,112,follow_VBG disbandment_NN destiny_NN child_NN ...,after_IN second_JJ solo_NN album_NN entertainm...
4,follow_VBG disbandment_NN destiny_NN child_NN ...,to_TO set_VB record_NN grammys_NN many_JJ beyo...,six_CD,1,7,112,follow_VBG disbandment_NN destiny_NN child_NN ...,to_TO set_VB record_NN grammys_NN many_JJ beyo...


In [21]:
batch_size=20
num_epochs=20
num_steps = 4



In [22]:
train_data_generator = lib.KerasBatchGenerator(train_data, num_steps=num_steps,
                                              batch_size=batch_size,
                                              vocabulary=vocabulary, 
                                              skip_step=df.question_len.max()+1)
valid_data_generator = lib.KerasBatchGenerator(valid_data, num_steps=num_steps,
                                              batch_size=batch_size,
                                              vocabulary=vocabulary, 
                                              skip_step=df.question_len.max()+1)


In [23]:
df.question_len.max()

22

In [24]:
hidden_size = embedding_dim
use_dropout=True
num_epochs=20

model_seq = Sequential([
    layers.Embedding(input_dim=vocabulary_size, 
                     output_dim=hidden_size,input_length=num_steps,
                     weights = [embedding_matrix],
                     mask_zero=True, trainable=True),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, dropout=0.25, return_sequences=True),
    layers.TimeDistributed(Dense(vocabulary_size)),
    layers.Activation('softmax')
])


optimizer=Adam()
model_seq.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model_seq.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 301)            38522883  
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 301)            726012    
_________________________________________________________________
lstm_5 (LSTM)                (None, 4, 301)            726012    
_________________________________________________________________
lstm_6 (LSTM)                (None, 4, 301)            726012    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 127983)         38650866  
_________________________________________________________________
activation_2 (Activation)    (None, 4, 127983)         0         
Total params: 79,351,785
Trainable params: 79,351,785
Non-trainable params: 0
________________________________________________________________

In [25]:
model_seq.fit_generator(train_data_generator.generate(one_sequence=True), steps_per_epoch = 30, 
                    epochs=num_epochs,
                    validation_data=valid_data_generator.generate(one_sequence=True),
                    validation_steps=20)

Epoch 1/120
30/30 [==============================] - 94s 3s/step - loss: 11.6525 - categorical_accuracy: 0.0133 - val_loss: 11.5450 - val_categorical_accuracy: 0.0075
Epoch 2/120
30/30 [==============================] - 88s 3s/step - loss: 11.6273 - categorical_accuracy: 0.0042 - val_loss: 12.5779 - val_categorical_accuracy: 0.0000e+00
Epoch 3/120
30/30 [==============================] - 88s 3s/step - loss: 11.7253 - categorical_accuracy: 0.0042 - val_loss: 11.0407 - val_categorical_accuracy: 0.0075
Epoch 4/120
30/30 [==============================] - 88s 3s/step - loss: 11.0059 - categorical_accuracy: 0.0083 - val_loss: 10.8665 - val_categorical_accuracy: 0.0200
Epoch 5/120
30/30 [==============================] - 90s 3s/step - loss: 10.2254 - categorical_accuracy: 0.0308 - val_loss: 11.2516 - val_categorical_accuracy: 0.0013
Epoch 6/120
30/30 [==============================] - 87s 3s/step - loss: 11.0087 - categorical_accuracy: 0.0042 - val_loss: 10.9824 - val_categorical_accuracy: 0

In [26]:
model_seq.save('model_seq_120ep_300dtrainable.h5')

In [35]:
model_seq_hist = model_seq.fit_generator(valid_data_generator.generate(one_sequence=True), epochs=10, steps_per_epoch=10)

# print(model_seq.history['accuracy'])

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/utils/data_utils.py:651: DeprecationWarning: `wait_time` is not used anymore.
  DeprecationWarning)


Epoch 1/10
10/10 [==============================] - 31s 3s/step - loss: 9.6868 - categorical_accuracy: 0.0100
Epoch 2/10
10/10 [==============================] - 30s 3s/step - loss: 9.7769 - categorical_accuracy: 0.0125
Epoch 3/10
10/10 [==============================] - 28s 3s/step - loss: 9.9558 - categorical_accuracy: 0.0150
Epoch 4/10
10/10 [==============================] - 28s 3s/step - loss: 10.0826 - categorical_accuracy: 0.0175
Epoch 5/10
10/10 [==============================] - 28s 3s/step - loss: 9.6149 - categorical_accuracy: 0.0150
Epoch 6/10
10/10 [==============================] - 28s 3s/step - loss: 9.8625 - categorical_accuracy: 0.0175
Epoch 7/10
10/10 [==============================] - 29s 3s/step - loss: 10.2852 - categorical_accuracy: 0.0125
Epoch 8/10
10/10 [==============================] - 31s 3s/step - loss: 10.1278 - categorical_accuracy: 0.0075
Epoch 9/10
10/10 [==============================] - 29s 3s/step - loss: 10.2136 - categorical_accuracy: 0.0300
Epoch 

TypeError: 'History' object is not subscriptable

In [39]:
print(model_seq.history.history['categorical_accuracy'])

[0.010000000149011612, 0.012500000186264515, 0.015000000223517418, 0.017500000447034834, 0.015000000223517418, 0.01750000026077032, 0.012500000186264515, 0.007500000111758709, 0.030000000819563867, 0.012500000186264515]
